In [ ]:
import os
from jinja2 import Template # type: ignore
from utils import takuid_new # type: ignore
from datetime import datetime, timedelta
import json
import zipfile

BASE = '/opt/wipbin/missions'
creator_uid = takuid_new()
mission_uid = takuid_new()

print(creator_uid)
print(mission_uid)

In [ ]:
mission_name = input("Mission Name: ")
mission_folder = os.path.join(BASE, mission_name)
os.makedirs(mission_folder, exist_ok=True)
mission = [
  {'aid_request': 1, 'name': 'Alpha', 'status': 'Completed', 'lat': 30.386165, 'lon': -97.927697, 'aid_address': '123 Alpha St'},
  {'aid_request': 2, 'name': 'Bravo', 'status': 'In Progress', 'lat': 30.387165, 'lon': -97.928697, 'aid_address': '456 Bravo St'},
  {'aid_request': 3, 'name': 'Charlie', 'status': 'Not Started', 'lat': 30.388165, 'lon': -97.929697, 'aid_address': '789 Charlie St'}
]
# add unique UID for each
mission = list(map(lambda mission: {**mission, 'uid': takuid_new()}, mission))
print(mission)
cot_type = 'a-u-G'
ce = '9999999.0'
le = '9999999.0'
hae = '9999999.0'

creator_type = 'a-f-G-U-C'
creator_name = 'informs1'
# human, geocoded, other, original for Azure maps
how = 'h-g-o-o'

created_time = (datetime.now() - timedelta(minutes=15)).strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3] + 'Z'
start_time = (datetime.now()).strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3] + 'Z'
stale_time = (datetime.now() + timedelta(days=180)).strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3] + 'Z'


cot_template = """
<?xml version='1.0' encoding='UTF-8' standalone='yes'?>
<event version='2.0' uid='{{ uid }}' type='{{ cot_type }}' time='{{ start_time }}' start='{{ start_time }}' stale='{{ stale_time }}' how='{{ how }}' access='Undefined'>
  <point lat='{{ lat }}' lon='{{ lon }}' hae='{{ hae }}' ce='{{ ce }}' le='{{ le }}' />
  <detail>
    <status readiness='true'/>
    <archive/>
    <contact callsign='{{ callsign }}'/>
    <remarks>{{ note }}</remarks>
    <color argb='-1'/>
    <link uid='{{ creator_uid }}' production_time='{{ created_time }}' type='{{ creator_type }}' parent_callsign='{{ creator_name }}' relation='p-p'/>
    <archive/>
    <precisionlocation geopointsrc='Azure'/>
  </detail>
</event>
"""
cot_template = cot_template.lstrip()
template = Template(cot_template)

In [18]:
for aid_request in mission:
  cot_file = template.render(
    aid_id=str(aid_request['aid_request']),
    name=aid_request['name'],
    cot_type = cot_type,
    status=aid_request['status'],
    uid=aid_request['uid'],
    start_time=start_time,
    stale_time=stale_time,
    how=how,
    ce=ce,
    hae=hae,
    le=le,
    lat=aid_request['lat'],
    lon=aid_request['lon'],
    callsign=f"AidRequest_{aid_request['aid_request']}",
    creator_uid=creator_uid,
    note="",
    created_time=created_time,
    creator_name=creator_name,
    creator_type=creator_type
  )
  # print(cot_file)
  event_folder = os.path.join(mission_folder, aid_request['uid'])
  os.makedirs(event_folder, exist_ok=True)
  file_path = os.path.join(event_folder, f"{aid_request['uid']}.cot")
  with open(file_path, 'w') as file:
    file.write(cot_file)


In [19]:
template_manifest = """
<?xml version="1.0" encoding="UTF-8"?>
<MissionPackageManifest version="2">
  <Configuration>
    <Parameter name="uid" value="{{ mission_uid }}"/>
    <Parameter name="name" value="{{ mission_name }}"/>
  </Configuration>
  <Contents>
    {%- for aid_request in aid_requests %}
    <Content ignore="false" zipEntry="{{ aid_request.uid }}/{{ aid_request.uid }}.cot">
      <Parameter name="uid" value="{{ aid_request.uid }}"/>
      <Parameter name="name" value="{{ aid_request.name }}"/>
    </Content>
    {%- endfor %}
  </Contents>
</MissionPackageManifest>
"""
template_manifest = template_manifest.lstrip()
# print(template_manifest)

In [20]:
template = Template(template_manifest)
manifest_file = template.render(
          mission_uid=mission_uid,
          mission_name=mission_name,
          aid_requests=mission
          )
manifest_folder = os.path.join(mission_folder, "MANIFEST")
os.makedirs(manifest_folder, exist_ok=True)
file_path = os.path.join(manifest_folder, "manifest.xml")
with open(file_path, 'w') as file:
  file.write(manifest_file)

In [32]:
def zip_folder(folder_path, zip_name):
    """
    Zips the contents of a folder into a named zip file.

    Args:
        folder_path (str): Path to the folder to be zipped.
        zip_name (str): Name of the resulting zip file (include .zip extension).
    """
    with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
      for root, dirs, files in os.walk(folder_path):
        for file in files:
          if file == '.DS_Store':
            continue
          file_path = os.path.join(root, file)
          arcname = os.path.relpath(file_path, folder_path)
          zipf.write(file_path, arcname)

In [ ]:

folder_to_zip = mission_folder
output_zip_name = os.path.join(BASE, f'{mission_name}.zip')

zip_folder(folder_to_zip, output_zip_name)
print(f'created: {output_zip_name}')

## Setting the How attribute

In Cursor-on-Target (CoT) messages, the how attribute describes the method by which the information was obtained, providing context about its origin and reliability. This attribute uses a hyphen-separated string, where each segment represents a specific aspect of the data acquisition process.

Common how Attribute Components:


```
Source of Information:

h: Human-generated
m: Machine-generated
s: Simulation
Positioning Method:

g: Geolocated
i: Instrumented
r: Radar
c: Calculated
Geolocation Technique:

g: GPS
i: INS (Inertial Navigation System)
l: LORAN
o: Other
Data Authenticity:

o: Original
d: Derived
f: Fused
Examples:

how='m-g-r-g-o': Machine-generated data, geolocated via radar using GPS, representing original information.

how='h-g-i-i-d': Human-generated data, geolocated using an instrument (INS), representing derived information.

Constructing the how Attribute for Specific Scenarios:

Azure Maps Geocoded Data from a Human-Provided Address:

Source: Human (h)
Positioning Method: Geocoded (g)
Geolocation Technique: Other (o) (since Azure Maps uses multiple techniques)
Data Authenticity: Original (o)
Resulting how Attribute: how='h-g-o-o'

Manually Entered Location:

Source: Human (h)
Positioning Method: Manual (m)
Geolocation Technique: Not applicable (n)
Data Authenticity: Original (o)
Resulting how Attribute: how='h-m-n-o'

These how attribute values provide clarity on how the location data was obtained, aiding in the assessment of its accuracy and reliability.

References:

For a comprehensive understanding of CoT message attributes and their usage, refer to the Cursor-on-Target Message Router User's Guide.

```